In [1]:
using Pkg
Pkg.activate("."); 
Pkg.instantiate()

  Activating new environment at `C:\Users\email\OneDrive\Dokumente\Uni\Köln\21WS\ATIS3\GroupProject\ATIES3_group_project-main\Project.toml`
  No Changes to `C:\Users\email\OneDrive\Dokumente\Uni\Köln\21WS\ATIS3\GroupProject\ATIES3_group_project-main\Project.toml`
  No Changes to `C:\Users\email\OneDrive\Dokumente\Uni\Köln\21WS\ATIS3\GroupProject\ATIES3_group_project-main\Manifest.toml`


In [13]:
using JuMP, Plots, CPLEX, DataFrames, XLSX, # IterTools, Statistics, StatsPlots, 

In [31]:
data = Dict()
data["time_series"] = DataFrame(XLSX.readtable("data_ATIS3.xlsx", "time_series")...)
data["scenarios"] = DataFrame(XLSX.readtable("data_ATIS3.xlsx", "scenarios")...) ;

Orientiert an C1_Producer_Pool.

\Omega = 100 Szenarien für jede betrachtete Periode t Element 1:T

Ich habe die Parameter jetzt nicht in einem Dict gespeichert, weil das das Aufrufen später weniger voluminös macht.

In [83]:
### Sets (or rather, set cardinalities)
# Number scenarios
Ω = 100
# Number time periods
T = 2

### Scalars
# Factors to FC that limit DA bids
FC_DA_fac_lo = 0.85
FC_DA_fac_up = 1.15
# Maximum power production
q_max = 100

### Vectors ###
## Length T
# Prices in DA, 15 and ID
p_DA  = data["time_series"][1:param[:T],"p_da"]
p_15  = data["time_series"][1:param[:T],"p_15"]
p_ID3 = data["time_series"][1:param[:T],"p_id3"]
# Power ForeCasted and real
q_FC = ones(T)
q_r = ones(T)
## Length Ω
# FC error sign; 1st stage decission
F_sgn = append!(-1 * ones(size(data["scenarios"].neg)), ones(size(data["scenarios"].pos)))
# Relative FC errors
q_ΔF = append!(data["scenarios"].neg, data["scenarios"].pos)
# Scenarios probablilities
pi = 1/param[:Ω] * ones(param[:Ω]);

In [76]:
wind = Model(CPLEX.Optimizer);

In [85]:
@variable(wind, q_DA[1:T] >= 0) # No ω dependence due to non-anticipativity!?
@variable(wind, q_15[1:T, 1:Ω]);
# q_ID is in fact not needed, as it only depends on q_DA, q_15 and q_r

LoadError: An object of name q_DA is already attached to this model. If this
    is intended, consider using the anonymous construction syntax, e.g.,
    `x = @variable(model, [1:N], ...)` where the name of the object does
    not appear inside the macro.

    Alternatively, use `unregister(model, :q_DA)` to first unregister
    the existing name from the model. Note that this will not delete the
    object; it will just remove the reference at `model[:q_DA]`.


In [82]:
@objective(wind, Max,
    sum(
        sum(
            p_DA[t]*q_DA[t] + p_15[t]*q_15[t, ω] + p_ID3[t]*(q_r[t]-q_DA[t]-q_15[t, ω])
        for ω in 1:Ω)
    for t in 1:T)
);

In [84]:
@constraint(wind, MaxPowerDA[t in 1:T],
    q_DA[t] <= q_max);

@constraint(wind, MaxPowerTot[t in 1:T, ω in 1:Ω],
    q_DA[t]+q_15[t, ω] <= q_max);



In [ ]:
### OLD: Save in Dict
par = Dict(
    ### Sets (or rather, set cardinalities)
    # Number scenarios
    :Ω => 100,
    # Number time periods
    :T => 2,
    
    ### Scalars
    # Factors to FC that limit DA bids
    :FC_DA_fac_lo => 0.85,
    :FC_DA_fac_up => 1.15,
    
    ### Vectors ###
    ## Length T
    # Prices in DA, 15 and ID
    :p_DA  => data["time_series"][1:param[:T],"p_da"],
    :p_15  => data["time_series"][1:param[:T],"p_15"],
    :p_ID3 => data["time_series"][1:param[:T],"p_id3"],
    # Power ForeCasted and real
    :q_FC => ones(T)
    :q_r => ones(T)
    ## Length Ω
    # FC error sign; 1st stage decission
    :F_sgn => append!(-1 * ones(size(data["scenarios"].neg)), ones(size(data["scenarios"].pos))),
    # Relative FC errors
    :q_ΔF => append!(data["scenarios"].neg, data["scenarios"].pos),
    # Scenarios probablilities
    :pi => 1/param[:Ω] * ones(param[:Ω]),
    
    
)

In [ ]:
### OLD: Stages kriegen einzelne Nummerierungen
par = Dict(
    ### Sets (or rather, set cardinalities)
    # Number scenarios first stage
    :Ω => 2,
    # Number scenarios sec stage
    :Φ => 50,
    # Number time periods
    :T => 2,
    
    ### Scalars
    # Factors to FC that limit DA bids
    :FC_DA_fac_lo => 0.85,
    :FC_DA_fac_up => 1.15,
    
    ### Vectors
    # Prices in DA, 15 and ID
    :p_DA  => data["time_series"][1:param[:T],"p_da"],
    :p_15  => data["time_series"][1:param[:T],"p_15"],
    :p_ID3 => data["time_series"][1:param[:T],"p_id3"],
    # Scenarios probablilities
    :pi_ω => 1/param[:Ω] * ones(param[:Ω]),
    :pi_ϕ => 1/param[:Φ] * ones(param[:Φ]),
    # 1st stage decission: Forecast sign
    :FC_sgn => [-1, +1],
    
    
);